In [ ]:
import json
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW

def train_model_with_mistral():
    """
    Trains a language model using the Mistral model for generating song lyrics based on user requests.
    
    This function performs the following steps:
    1. Loads the Mistral tokenizer and model.
    2. Loads the data from a JSON file.
    3. Extracts the user requests and assistant lyrics from the data.
    4. Concatenates the request with the lyrics to form the input text.
    5. Tokenizes the input text.
    6. Trains the model using the tokenized input.
    7. Saves the trained model.
    
    Returns:
    None
    """
    
    # Load the Mistral tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
    
    # Add padding token to the tokenizer
    tokenizer.pad_token = tokenizer.eos_token
    
    # Load the data from the JSON file
    with open("train.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Extract the user requests and assistant lyrics
    requests = [item["content"] for item in data if item["role"] == "user"]
    lyrics = [item["content"] for item in data if item["role"] == "assistant"]
    
    # Concatenate the request with the lyrics to form the input
    input_text = [f"{request}: {lyric}" for request, lyric in zip(requests, lyrics)]
    
    # Tokenize the input data
    max_length = 128  # Set a maximum length for truncation
    tokenized_input = tokenizer(input_text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    
    # Set the model to training mode
    model.train()
    
    # Configure the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    
    # Training loop
    epochs = 3
    batch_size = 4
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for i in range(0, len(tokenized_input["input_ids"]), batch_size):
            batch_inputs = {
                key: value[i:i+batch_size].to(model.device) for key, value in tokenized_input.items()
            }
            optimizer.zero_grad()
            outputs = model(**batch_inputs, labels=batch_inputs["input_ids"])
            loss = outputs.loss
            loss.backward()
            optimizer.step()
    
            print(f"Batch {i//batch_size + 1}/{len(tokenized_input['input_ids'])//batch_size}, Loss: {loss.item():.4f}")
    
    # Save the trained model
    model.save_pretrained(r"\lyrics_generation_model_mistral", save_format="pytorch")
    
